In [40]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Load dataset
df = pd.read_csv("mnist_train.csv")  # Replace with your path

# Separate features and labels
X = df.drop('label', axis=1).values / 255.0  # Normalize
y = to_categorical(df['label'].values, num_classes=10)  # One-hot encode

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [41]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Create the model
model = Sequential()
model.add(Dense(128, input_shape=(784,), activation='sigmoid'))  # Hidden layer with Sigmoid
model.add(Dense(64, activation='sigmoid'))  # Another hidden layer with Sigmoid
model.add(Dense(10, activation='sigmoid'))  # Output layer with Sigmoid (Multi-label style)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10


D:\pythonAllcode\project1\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1200/1200 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.5999 - loss: 0.2507 - val_accuracy: 0.9139 - val_loss: 0.0620
Epoch 2/10
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9198 - loss: 0.0548 - val_accuracy: 0.9365 - val_loss: 0.0410
Epoch 3/10
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9460 - loss: 0.0358 - val_accuracy: 0.9491 - val_loss: 0.0327
Epoch 4/10
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9586 - loss: 0.0277 - val_accuracy: 0.9549 - val_loss: 0.0275
Epoch 5/10
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9667 - loss: 0.0223 - val_accuracy: 0.9591 - val_loss: 0.0250
Epoch 6/10
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9748 - loss: 0.0176 - val_accuracy: 0.9649 - val_loss: 0.0219
Epoch 7/10
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9778 - loss: 0.0150 - val_accuracy: 0.9656 - val_loss: 0.0210
Epoch 8/10
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9847 - loss: 0.0118 - val_accurac

In [42]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

375/375 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9736 - loss: 0.0181
Test Loss: 0.018301019445061684
Test Accuracy: 0.9730833172798157


In [46]:
y_pred_prob = model.predict(X_test)                       # Probabilities
y_pred = np.argmax(y_pred_prob, axis=1)
y_true = np.argmax(y_test, axis=1)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Example: y_true = [actual labels], y_pred = [predicted labels]
print("Accuracy:", accuracy_score(y_true, y_pred))
print("Precision:", precision_score(y_true, y_pred, average='weighted'))
print("Recall:", recall_score(y_true, y_pred, average='weighted'))
print("F1 Score:", f1_score(y_true, y_pred, average='weighted'))
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))

375/375 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Accuracy: 0.9730833333333333
Precision: 0.9733705946777979
Recall: 0.9730833333333333
F1 Score: 0.9730863588307693
Confusion Matrix:
 [[1156    0    4    0    3    2    3    2    4    1]
 [   0 1301   11    3    1    0    1    3    0    2]
 [   1    3 1153    2    5    0    1    5    3    1]
 [   0    1   21 1167    1    9    2    7    7    4]
 [   1    1    1    0 1161    1    3    5    0    3]
 [   7    1    5   10    5 1058    6    4    6    2]
 [   3    0    7    0    2    3 1160    0    2    0]
 [   0    6   12    1    6    0    0 1272    1    1]
 [   1    3   11    6    5    5    3    2 1122    2]
 [   3    2    0    5   29    7    0   15    6 1127]]


In [44]:
model.save('mnist_digit_customNN_sigmoid_better.h5')

In [31]:
import ipywidgets as widgets
from IPython.display import display
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from PIL import Image
import io
import matplotlib.pyplot as plt

# Load your trained model
model = load_model("mnist_digit_model.h5")  # Change to your actual model path

# Upload widget
upload = widgets.FileUpload(accept='.jpg,.png,.jpeg', multiple=False)
display(upload)

# Button and output area
button = widgets.Button(description="Predict")
output = widgets.Output()
display(button, output)

def preprocess_and_predict(image_data, model):
    # Step 1: Load and convert the image to grayscale
    image = Image.open(io.BytesIO(image_data)).convert('L')
    original_img = np.array(image)

    # Resize if the image is too large
    if max(original_img.shape) > 300:
        original_img = cv2.resize(original_img, (300, 300))

    # Step 2: Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(original_img, (5, 5), 0)

    # Step 3: Apply adaptive thresholding for better contrast and binarization
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY_INV, 11, 3)

    # Step 4: Find external contours (digits)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        print("❌ No digits found.")
        return "", original_img, []

    # Step 5: Sort contours from left to right based on their x-coordinate
    bounding_boxes = [cv2.boundingRect(c) for c in contours]
    sorted_boxes = sorted(bounding_boxes, key=lambda b: b[0])

    digits = []
    predictions = []

    for (x, y, w, h) in sorted_boxes:
        # Skip too-small contours (likely noise)
        if w < 5 or h < 5:
            continue

        # Step 6: Extract the region of interest (ROI) from the thresholded image
        roi = thresh[y:y+h, x:x+w]

        # Step 7: Resize the ROI to 20x20, keeping the aspect ratio
        if h > w:
            new_h = 20
            new_w = int(w * (20 / h))
        else:
            new_w = 20
            new_h = int(h * (20 / w))

        # Ensure the resize maintains a proper aspect ratio
        resized_digit = cv2.resize(roi, (new_w, new_h))

        # Step 8: Pad the resized digit to 28x28 (standard MNIST size)
        # Use a smaller padding amount to prevent large black borders.
        padded = np.pad(resized_digit,
                        (((28 - new_h) // 2, (28 - new_h + 1) // 2),
                         ((28 - new_w) // 2, (28 - new_w + 1) // 2)),
                        mode='constant', constant_values=0)

        # Step 9: Normalize the padded image to [0, 1] and flatten it
        normalized = padded / 255.0
        flattened = normalized.reshape(1, 784)

        # Step 10: Make the prediction
        pred = model.predict(flattened)
        digit = np.argmax(pred)
        predictions.append(str(digit))
        digits.append(padded)

    # Step 11: Combine the individual predictions to form the full number
    full_number = ''.join(predictions)
    return full_number, original_img, digits





def on_button_clicked(b):
    with output:
        output.clear_output()
        if upload.value:
            file_data = upload.value[0]
            image_bytes = file_data['content']

            number, original_img, digit_imgs = segment_and_predict(image_bytes, model)

            # Show original + segmented digits
            fig, axes = plt.subplots(1, len(digit_imgs) + 1, figsize=(10, 4))
            axes[0].imshow(original_img, cmap='gray')
            axes[0].set_title("Original Image")
            axes[0].axis('off')

            for i, img in enumerate(digit_imgs):
                axes[i + 1].imshow(img, cmap='gray')
                axes[i + 1].set_title(f"Digit {i+1}")
                axes[i + 1].axis('off')

            plt.suptitle(f"✅ Predicted Number: {number}", fontsize=14)
            plt.tight_layout()
            plt.show()
        else:
            print("❌ Please upload an image first.")



button.on_click(on_button_clicked)


FileUpload(value=(), accept='.jpg,.png,.jpeg', description='Upload')

Button(description='Predict', style=ButtonStyle())

Output()

In [38]:
# Create the model
model1 = Sequential()
model1.add(Dense(128, input_shape=(784,), activation='relu'))  # Hidden layer with ReLU
model1.add(Dense(64, activation='relu'))  # Another hidden layer with ReLU
model1.add(Dense(10, activation='softmax'))  # Output layer with softmax for classification (Multi-class)
# Compile the model
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model1.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

D:\pythonAllcode\project1\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.7982 - loss: 0.1261 - val_accuracy: 0.9510 - val_loss: 0.0322
Epoch 2/10
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9572 - loss: 0.0288 - val_accuracy: 0.9643 - val_loss: 0.0232
Epoch 3/10
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9726 - loss: 0.0185 - val_accuracy: 0.9688 - val_loss: 0.0208
Epoch 4/10
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9804 - loss: 0.0136 - val_accuracy: 0.9716 - val_loss: 0.0186
Epoch 5/10
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9861 - loss: 0.0098 - val_accuracy: 0.9728 - val_loss: 0.0182
Epoch 6/10
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9899 - loss: 0.0075 - val_accuracy: 0.9751 - val_loss: 0.0166
Epoch 7/10
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9919 - loss: 0.0060 - val_accuracy: 0.9734 - val_loss: 0.0187
Epoch 8/10
1200/1200 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9942 - loss: 0.0048 - 

In [39]:
model.save('mnist_digit_customNN_reluSoftmax_best.h5')

In [32]:
import psutil

mem = psutil.virtual_memory()
print(f"Total: {mem.total / (1024**3):.2f} GB")
print(f"Used: {mem.used / (1024**3):.2f} GB")
print(f"Free: {mem.available / (1024**3):.2f} GB")


Total: 7.88 GB
Used: 5.83 GB
Free: 2.05 GB
